<a href="https://colab.research.google.com/github/Bouaskaoun/img_classification/blob/main/img_classification_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
file_name = '/content/images_tifinagh.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [27]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io as io
from skimage import color
from skimage.transform import resize
import shutil

In [81]:
output_num = [];
images = [];
flaten_images = [];
alphabet = [];
data_path = '/content/images_tifinagh/';
category_path = '/content/category/'

for files in os.listdir(data_path + '/00'):
  alpha = files[:-4]
  alphabet.append(alpha.lower())
  alphabet.sort()

sub_folders = os.listdir(data_path)
sub_folders.sort()

In [82]:
alphabet = os.listdir(category_path)
alphabet.sort()

In [29]:
for alpha in alphabet:
    path = os.path.join(category_path, alpha)
    os.mkdir(path)

In [30]:
for folder in sub_folders:
  folder = os.path.join(data_path,folder)
  for img in os.listdir(folder):
    if folder[-2:] == '00':
      index = -4
    elif folder[-2:] in sub_folders[10:41]:
      index = -6
    elif folder[-2:] in sub_folders[1:10]:
      index = -5
    extension = img[:index].lower()
    original = folder+'/'+img
    target = category_path+extension+'/'+img
    shutil.move(original, target)

In [ ]:
for cat in alphabet:
    class_num = alphabet.index(cat)
    path = os.path.join(category_path,cat)
    for img in os.listdir(path):
        img_path = os.path.join(path,img)
        img_array = io.imread(img_path)
        grayscale = color.rgb2gray(img_array)
        img_shape = img_array.shape
        #print(img_shape)
        img_resized = resize(grayscale,(30,30,1))
        images.append(img_resized)
        flaten_images.append(img_resized.flatten())
        output_num.append(class_num)

In [50]:
flaten_images = np.array(flaten_images)
output_num = np.array(output_num)
images = np.array(images)

In [ ]:
output_num

In [ ]:
np.unique(output_num,return_counts=True)

In [53]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(flaten_images, output_num, test_size=0.3, random_state=109)

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
param_grid = [
              {'C':[1, 10, 100, 1000], 'kernel':['linear']},
              {'C':[1, 10, 100, 1000], 'gamma':[0.001, 0.0001], 'kernel':['rbf']},
              ]
svc = svm.SVC(probability=True)
clf = GridSearchCV(svc, param_grid)
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
y_pred

In [57]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [60]:
import pickle
pickle.dump(clf,open('img_model.p','wb'))

In [61]:
model = pickle.load(open('img_model.p','rb'))

In [ ]:
flat_data = []
url = '/content/category/ww/ww12.jpg'
img = io.imread(url)
img_array = color.rgb2gray(img)
img_resized = resize(img_array,(30,30,1))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
plt.imshow(img)
y_out = model.predict(flat_data)
print(y_out)
y_out = alphabet[y_out[0]]
print(f'predicted output : {y_out}')

In [24]:
!rm -rf images_tifinagh

In [25]:
!rm -rf category